In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('2015_sample.csv')
df = pd.concat([df,pd.read_csv('2015_failures.csv')])
df.head()

,date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,2015-02-11,MJ0351YNG9VAJA,Hitachi HDS5C3030ALA630,3.000593e+12,0,100.0,0.0,135.0,108.0,141.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-03-16,Z3016SZJ,ST4000DM000,4.000787e+12,0,116.0,113638200.0,NaN,NaN,93.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-12-01,W300CDRJ,ST4000DM000,4.000787e+12,0,119.0,210314536.0,NaN,NaN,91.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-02-09,W300AWBZ,ST4000DM000,4.000787e+12,0,117.0,167638104.0,NaN,NaN,95.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-03-03,MJ1311YNG6XKGA,Hitachi HDS5C3030ALA630,3.000593e+12,0,100.0,0.0,135.0,108.0,140.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df_norm = df[['smart_1_normalized', 'smart_3_normalized', 'smart_4_normalized', 'smart_5_normalized',
           'smart_7_normalized','smart_12_normalized', 'smart_194_normalized', 
           'smart_197_normalized', 'smart_198_normalized', 'smart_199_normalized']]
df_norm.sample(n=5)

,smart_1_normalized,smart_3_normalized,smart_4_normalized,smart_5_normalized,smart_7_normalized,smart_12_normalized,smart_194_normalized,smart_197_normalized,smart_198_normalized,smart_199_normalized
1241,100.0,122.0,100.0,100.0,100.0,100.0,253.0,100.0,100.0,200.0
320,105.0,92.0,100.0,98.0,79.0,100.0,33.0,97.0,97.0,200.0
1044,119.0,97.0,100.0,100.0,85.0,100.0,29.0,100.0,100.0,200.0
1171,120.0,91.0,100.0,94.0,89.0,100.0,28.0,100.0,100.0,200.0
786,119.0,91.0,100.0,99.0,89.0,100.0,24.0,100.0,100.0,200.0


In [10]:
X = df_norm.dropna(0,thresh =10).dropna(1)
y = df['failure'].values

In [11]:
y.shape

(2856,)

In [13]:
import lightgbm as lgb
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, mean_absolute_error,accuracy_score
from sklearn.metrics import classification_report

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [39]:
y_val

array([0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,

In [ ]:
y_train

In [42]:
eval_set = lgb.Dataset(X_val, label=y_val)
param = {'objective':'binary_logloss'}
train_data = lgb.Dataset(X_train, label=y_train, params=param)

num_round = 10
bst = lgb.train(params=param, train_set=train_data, num_boost_round=10, valid_sets=[eval_set])

LightGBMError: Unknown objective type name: binary-logloss

In [40]:
y_pred = bst.predict(X_test)
print("Classification Report: \n", classification_report(y_test, y_pred))
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
accuracy = accuracy_score(y_test, y_pred)
print("\nAccuracy",accuracy)

ValueError: Mix type of y not allowed, got types {'continuous', 'binary'}

In [41]:
y_pred

array([0.38182779, 0.81232894, 0.57184301, 0.24571337, 0.42215589,
       0.81232894, 0.6520011 , 0.25433684, 0.27584101, 0.57787945,
       0.32930799, 0.42775562, 0.58985403, 0.52019513, 0.57787945,
       0.60079724, 0.74679889, 0.67886624, 0.24571337, 0.4399132 ,
       0.4823489 , 0.81232894, 0.56004556, 0.2759553 , 0.41271355,
       0.44560694, 0.49907354, 0.56157101, 0.27584101, 0.41510507,
       0.42775562, 0.81232894, 0.56468654, 0.42775562, 0.81232894,
       0.59004218, 0.81232894, 0.27584101, 0.50281369, 0.47556215,
       0.59523841, 0.38573268, 0.49745779, 0.45114758, 0.41271355,
       0.65604243, 0.49274071, 0.27584101, 0.46246765, 0.30429316,
       0.5231939 , 0.57621085, 0.5331862 , 0.31650631, 0.53619685,
       0.2950779 , 0.69150475, 0.49536548, 0.50411803, 0.50554513,
       0.52019513, 0.81232894, 0.53843962, 0.41271355, 0.5012763 ,
       0.49404002, 0.47681171, 0.73360628, 0.56940114, 0.61265306,
       0.65604243, 0.81232894, 0.54128251, 0.52429863, 0.37032

In [23]:
help(lgb.train)

Help on function train in module lightgbm.engine:

train(params, train_set, num_boost_round=100, valid_sets=None, valid_names=None, fobj=None, feval=None, init_model=None, feature_name='auto', categorical_feature='auto', early_stopping_rounds=None, evals_result=None, verbose_eval=True, learning_rates=None, keep_training_booster=False, callbacks=None)
    Perform the training with given parameters.
    
    Parameters
    ----------
    params : dict
        Parameters for training.
    train_set : Dataset
        Data to be trained.
    num_boost_round: int, optional (default=100)
        Number of boosting iterations.
    valid_sets: list of Datasets or None, optional (default=None)
        List of data to be evaluated during training.
    valid_names: list of string or None, optional (default=None)
        Names of ``valid_sets``.
    fobj : callable or None, optional (default=None)
        Customized objective function.
    feval : callable or None, optional (default=None)
        C

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
plt.plot(tpr,fpr)
plt.show() 